In [61]:
import torch
from torch import nn
from tqdm import tqdm

In [ ]:
# %load_ext nb_mypy
%nb_mypy DebugOff

In [69]:
def conv_output_size(w, k=3, p=2, s=1):
    return ((w - k + 2*p) / s + 1)

def conv_block(img_size, n_in, nh, kernel_size, stride, padding, max_kernel=2):
    modules = [
    nn.Conv2d(n_in, nh, kernel_size, stride, padding),
    nn.BatchNorm2d(nh),
    nn.ReLU(),
    nn.MaxPool2d(max_kernel)
    ]
    output_size = conv_output_size(img_size, k=kernel_size, p=padding, s=stride)
    output_size = conv_output_size(output_size, k=max_kernel, p=0, s=max_kernel)
    return modules, int(output_size)

class CustomModel(nn.Module):
    def __init__(self, n_in, n_out, nh, img_size, num_blocks, kernel_size=3, stride=1, padding=1):
        super(CustomModel, self).__init__()
        
        modules_list = []

        for n in range(num_blocks):
            modules, output_size = conv_block(img_size, n_in, nh, kernel_size, stride, padding)
            modules_list.extend(modules)
            
            img_size = output_size
            n_in = nh
            nh = 2*nh
        self.conv = nn.Sequential(*modules_list)
        output_size = output_size**2 * n_in

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(output_size, n_out)
        )
    
    def forward(self, x):
        x = self.conv(x)
        out = self.fc(x)
        return out
    
net = CustomModel(img.size(1), 6, 16, 124, 4)



Program before typechecking:
from IPython import get_ipython
import torch
from torch import nn
from tqdm import tqdm
pass
pass
pass
pass
import matplotlib.pyplot as plt


def plot_img(img):
    pass
def conv_output_size(w, k=3, p=2, s=1):
    return ((w - k + 2*p) / s + 1)

def conv_block(img_size, n_in, nh, kernel_size, stride, padding, max_kernel=2):
    modules = [
    nn.Conv2d(n_in, nh, kernel_size, stride, padding),
    nn.BatchNorm2d(nh),
    nn.ReLU(),
    nn.MaxPool2d(max_kernel)
    ]
    output_size = conv_output_size(img_size, k=kernel_size, p=padding, s=stride)
    output_size = conv_output_size(output_size, k=max_kernel, p=0, s=max_kernel)
    return modules, int(output_size)

class CustomModel(nn.Module):
    def __init__(self, n_in, n_out, nh, img_size, num_blocks, kernel_size=3, stride=1, padding=1):
        super(CustomModel, self).__init__()
        
        modules_list = []

        for n in range(num_blocks):
            modules, output_size = conv_block(img_size,

In [64]:
import matplotlib.pyplot as plt
def plot_img(img):
    plt.imshow(img.permute(1, 2, 0))

In [54]:
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
from torchvision import transforms
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, data_path, transform=[]):
        self.data_path = data_path
        self.transform = transform
        self.total_imgs = sorted([p for p in Path(data_path).rglob("*.jpg") if p.is_file()])
        classes = [p.name for p in Path(data_path).glob("*")]
        self.classes_to_label = dict(zip(classes, range(len(classes))))

    def __len__(self):
        return len(self.total_imgs)
    
    def __getitem__(self, idx):
        image = Image.open(self.total_imgs[idx])
        if self.transform:
            image = transforms.Compose(self.transform)(image)
        label = self.total_imgs[idx].parent.name
        class_label = self.classes_to_label[label]
        return image, class_label

transform = [transforms.Resize((124, 124)), transforms.ToTensor()]
data_path = "../tmp_files/seg_train/seg_train/"
train_ds = CustomDataset(data_path=data_path, transform=transform)


In [58]:
from sklearn.model_selection import train_test_split
val_split = 0.2
   
# train_indices, valid_indices = train_test_split(range(len(train_ds)), stratify=train_ds.)
train_ds[:20]


AttributeError: 'list' object has no attribute 'read'

In [126]:
def conv_output_size(w, k=3, p=2, s=1):
    return ((w - k + 2*p) / s + 1)
conv_output_size(124,3,1,1)
conv_output_size(124, 2, 0, 2)
62*62*16
# conv_output_size(1984, 1, 2)

61504

In [110]:
124*124*16

246016

In [72]:
def prime_factors(n):
    i = 2
    factors = []
    while i * i <= n:
        if n % i:
            i += 1
        else:
            n //= i
            factors.append(i)
    if n > 1:
        factors.append(n)
    return factors
prime_factors(63504)


[2, 2, 2, 2, 3, 3, 3, 3, 7, 7]

In [81]:
7*7*3

147